In [1]:
import sys; sys.path.append('../util')
from load_yelp_data import load_yelp_dataframe, restaurants_and_bars_in, train_test_split_reviews
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics
import scipy.optimize
%matplotlib inline

from baseline_calculator import *

In [2]:
businesses = load_yelp_dataframe('businesses')
reviews = load_yelp_dataframe('reviews')
users = load_yelp_dataframe('users')

In [3]:
phoenix_restaurants, phoenix_reviews, phoenix_users = restaurants_and_bars_in('Phoenix', businesses, reviews, users)

In [4]:
reviews_train, reviews_test = train_test_split_reviews(phoenix_reviews)

In [5]:
bc = L2RegLeastSquaresBaselineCalculator()
bc.fit(reviews_train)

828243.574404
820873.878045
814394.684473
808624.424522
803428.02802
798703.626399
794373.295229
790376.562701
786665.822104
783203.062639
779957.518782
776903.964011
774021.459808
771292.428842
768701.960885
766237.287241
763887.378337
761642.632134
759494.630209
757435.944728
755459.984103
753560.868332
751733.327358
749972.617459
748274.451873
746634.942808
745050.552587
743518.052239
742034.486165
740597.141817
739203.523547
737851.329934
736538.43404
735262.866145
734022.798586
732816.532403
731642.485523
730499.182288
729385.244127
728299.381244
727240.385173
726207.122103
725198.526878
724213.597585
723251.390671
722311.016518
721391.63544
720492.45403
719612.721849
718751.728401
717908.800365
717083.299075
716274.618193
715482.181592
714705.441395
713943.876178
713196.989312
712464.307434
711745.379028
711039.773123
710347.078073
709666.90044
708998.863944
708342.608498
707697.789301
707064.075999
706441.151898
705828.713238
705226.468501
704634.137781
704051.452177
703478.1532

In [6]:
bc.baseline_rmse(reviews_train)

1.2242238409059312

In [7]:
bc.baseline_rmse(reviews_test)

1.2585870968573407

In [8]:
bc2 = L2RegLeastSquaresBaselineCalculator()
bc2.fit(reviews_train, l2_pen=0.1)

828243.574404
820860.237999
814343.119102
808515.925852
803247.342421
798438.234688
794012.719585
789911.887563
786089.356487
782508.096981
779138.145874
775954.944036
772938.116315
770070.566836
767337.801075
764727.412346
762228.68858
759832.307888
757530.100308
755314.859346
753180.191363
751120.394007
749130.357163
747205.481515
745341.611043
743534.976606
741782.148471
740079.996078
738425.653735
736816.491199
735250.088301
733724.212966
732236.802076
730785.944736
729369.867587
727986.921873
726635.571999
725314.38539
724022.023463
722757.23358
721518.84184
720305.746626
719116.91279
717951.366424
716808.190132
715686.518749
714585.535466
713504.468301
712442.586894
711399.199576
710373.650701
709365.318197
708373.611325
707397.968625
706437.856023
705492.765089
704562.211437
703645.733243
702742.889875
701853.260631
700976.443562
700112.054388
699259.725486
698419.104951
697589.855726
696771.654783
695964.192369
695167.171294
694380.306273
693603.323304
692835.959091
692077.9605

In [9]:
bc2.baseline_rmse(reviews_train)

1.2183229084612857

In [10]:
bc2.baseline_rmse(reviews_test)

1.2554672409037799

In [11]:
bc2 = L2RegLeastSquaresBaselineCalculator()
bc2.fit(reviews_train, l2_pen=0.01, learn_rate=0.0001)

828243.574404
778705.977102
752820.672045
734938.756158
721236.507803
710107.434098
700717.621687
692578.403093
685379.314526
678911.32329
673027.38207
667620.421048
662610.26555
657935.459961
653547.947306
649409.490366
645489.196604
641761.765656
638206.222847
634804.987395
631543.175807
628408.073546
625388.729027
622475.637857
619660.494505
616935.995007
614295.678701
611733.80014
609245.224571
606825.341973
604469.99584
602175.423785
599938.207677
597755.231522
595623.6457
593540.836419
591504.3995
589512.117771
587561.941486
585651.971289
583780.443332
581945.716231
580146.259583
578380.643826
576647.531261
574945.668062
573273.877167
571631.051916
570016.150346
568428.190075
566866.243682
565329.434541
563816.933053
562327.953222
560861.74955
559417.614206
557994.874442
556592.890234
555211.052123
553848.779231
552505.517441
551180.73772
549873.934579
548584.624646
547312.34535
546056.653706
544817.125181
543593.35265
542384.945427
541191.528356
540012.740968
538848.236704
53769

In [12]:
bc2.baseline_rmse(reviews_train)

1.0599216476143467

In [13]:
bc2.baseline_rmse(reviews_test)

1.2265731676769089

In [26]:
class DecoupledRegularizedBaselineCalculator(BaselineCalculator):
    def fit(self, reviews, busi_reg_strength=2.5, user_reg_strength=5):
        y = reviews.stars.values
        u = reviews.user_id.values
        b = reviews.business_id.values
        mu = np.mean(y)
        self.global_mean = mu
        yu = defaultdict(list)
        yb = defaultdict(list)
        for i, star in enumerate(y):
            yb[b[i]].append(star)
            yu[u[i]].append((star, b[i]))
        for busi, stars in yb.items():
            self.busi_baselines[busi] = (
                sum(star - mu for star in stars) /
                float(busi_reg_strength + len(stars)))
        for user, star_biz_pairs in yu.items():
            self.user_baselines[user] = (
                sum(star - mu - self.busi_baselines[biz] for star, biz in star_biz_pairs) /
                float(user_reg_strength + len(star_biz_pairs)))



In [27]:
drbc = DecoupledRegularizedBaselineCalculator()
drbc.fit(reviews_train)

In [28]:
drbc.baseline_rmse(reviews_train)

1.0780053806249488

In [29]:
drbc.baseline_rmse(reviews_test)

1.2246467996995585

In [19]:
drbc = DecoupledRegularizedBaselineCalculator()

user_strs = [0.1, 1, 10, 25, 50]
busi_strs = [0.1, 1, 10, 25, 50]
for us in user_strs:
    for bs in busi_strs:
        drbc.fit(reviews_train, busi_reg_strength=bs, user_reg_strength=us)
        print('for {} usr, {} biz:'.format(us, bs))
        print('  train RMSE: {:.5f}, test RMSE: {:.5f}'.format(
                drbc.baseline_rmse(reviews_train),
                drbc.baseline_rmse(reviews_test)))

for 0.1 usr, 0.1 biz:
  train RMSE: 0.90397, test RMSE: 1.28577
for 0.1 usr, 1 biz:
  train RMSE: 0.90415, test RMSE: 1.28433
for 0.1 usr, 10 biz:
  train RMSE: 0.91111, test RMSE: 1.29098
for 0.1 usr, 25 biz:
  train RMSE: 0.91962, test RMSE: 1.30210
for 0.1 usr, 50 biz:
  train RMSE: 0.92871, test RMSE: 1.31472
for 1 usr, 0.1 biz:
  train RMSE: 0.95898, test RMSE: 1.24659
for 1 usr, 1 biz:
  train RMSE: 0.95921, test RMSE: 1.24446
for 1 usr, 10 biz:
  train RMSE: 0.96695, test RMSE: 1.24797
for 1 usr, 25 biz:
  train RMSE: 0.97630, test RMSE: 1.25672
for 1 usr, 50 biz:
  train RMSE: 0.98621, test RMSE: 1.26721
for 10 usr, 0.1 biz:
  train RMSE: 1.12176, test RMSE: 1.23001
for 10 usr, 1 biz:
  train RMSE: 1.12237, test RMSE: 1.22742
for 10 usr, 10 biz:
  train RMSE: 1.13330, test RMSE: 1.23000
for 10 usr, 25 biz:
  train RMSE: 1.14569, test RMSE: 1.23828
for 10 usr, 50 biz:
  train RMSE: 1.15847, test RMSE: 1.24838
for 25 usr, 0.1 biz:
  train RMSE: 1.16626, test RMSE: 1.23725
for 25 

In [25]:
drbc = DecoupledRegularizedBaselineCalculator()

user_strs = [2, 3, 4, 5, 6]
busi_strs = [2.5]
for us in user_strs:
    for bs in busi_strs:
        drbc.fit(reviews_train, busi_reg_strength=bs, user_reg_strength=us)
        print('for {} usr, {} biz:'.format(us, bs))
        print('  train RMSE: {:.5f}, test RMSE: {:.5f}'.format(
                drbc.baseline_rmse(reviews_train),
                drbc.baseline_rmse(reviews_test)))

for 2 usr, 2.5 biz:
  train RMSE: 1.00781, test RMSE: 1.23127
for 3 usr, 2.5 biz:
  train RMSE: 1.03914, test RMSE: 1.22685
for 4 usr, 2.5 biz:
  train RMSE: 1.06134, test RMSE: 1.22517
for 5 usr, 2.5 biz:
  train RMSE: 1.07801, test RMSE: 1.22465
for 6 usr, 2.5 biz:
  train RMSE: 1.09107, test RMSE: 1.22468
